<a href="https://colab.research.google.com/github/AndrewKostin/TUII_2019/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     |████████████████████████████████| 64.4MB 47kB/s 


In [2]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, KFold

from google.colab import files, drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/colab/ONTI_FINAL/{}'

# train = pd.read_csv(path.format('transactions_train.csv'))
# test = pd.read_csv(path.format('transactions_test.csv'))

target = pd.read_csv(
    path.format('train_target.csv'),
    index_col='client_dk')
target.columns = target.columns.astype(int)

test = pd.read_csv(
    path.format('test.csv'),
    index_col='client_dk')

big = pd.read_csv(
    path.format('big.csv'),
    index_col='client_dk')
# transactions = pd.concat([train, test]).reset_index(drop=True)

In [0]:
i_l, i_v = train_test_split(target.index, test_size=0.2, random_state=666)

In [8]:
clf = CatBoostClassifier(
    depth=6,
    learning_rate = 0.007,
    iterations=15000,
    verbose=200,
    use_best_model=True,
    random_state=430,
    eval_metric='AUC',
    custom_metric=['AUC'],
    task_type='GPU',
)

clf.fit(
    big.loc[i_l], target.loc[i_l, 27],
    eval_set=Pool(big.loc[i_v], target.loc[i_v, 27]),
    early_stopping_rounds = 700
)

0:	learn: 0.8496526	test: 0.8525985	best: 0.8525985 (0)	total: 67.8ms	remaining: 16m 56s
200:	learn: 0.8773266	test: 0.8710305	best: 0.8710814 (197)	total: 12.4s	remaining: 15m 15s
400:	learn: 0.8823030	test: 0.8719633	best: 0.8719947 (399)	total: 24.8s	remaining: 15m 1s
600:	learn: 0.8865522	test: 0.8726981	best: 0.8727129 (584)	total: 37.1s	remaining: 14m 49s
800:	learn: 0.8901244	test: 0.8731755	best: 0.8732108 (784)	total: 49.5s	remaining: 14m 36s
1000:	learn: 0.8932383	test: 0.8733718	best: 0.8734089 (992)	total: 1m 1s	remaining: 14m 23s
1200:	learn: 0.8961636	test: 0.8734118	best: 0.8735145 (1101)	total: 1m 13s	remaining: 14m 9s
1400:	learn: 0.8987667	test: 0.8735595	best: 0.8735675 (1380)	total: 1m 25s	remaining: 13m 53s
1600:	learn: 0.9013434	test: 0.8735224	best: 0.8736302 (1476)	total: 1m 38s	remaining: 13m 40s
1800:	learn: 0.9037625	test: 0.8737658	best: 0.8737658 (1800)	total: 1m 50s	remaining: 13m 27s
2000:	learn: 0.9060325	test: 0.8737697	best: 0.8738090 (1850)	total: 2m 

In [0]:
clfs = {}
clfs[27] = clf

In [10]:
clfs

{27: <catboost.core.CatBoostClassifier at 0x7f20e7d97940>}

In [14]:

for col in target.columns[1:]:
    clf_ = CatBoostClassifier(
        depth=6,
        learning_rate = 0.007,
        iterations=15000,
        verbose=250,
        use_best_model=True,
        random_state=430,
        eval_metric='AUC',
        custom_metric=['AUC'],
        task_type='GPU',
    )

    clf_.fit(
        big.loc[i_l], target.loc[i_l, col],
        eval_set=Pool(big.loc[i_v], target.loc[i_v, col]),
        early_stopping_rounds = 700
    )

    clfs[col] = clf_


0:	learn: 0.7598013	test: 0.7532011	best: 0.7532011 (0)	total: 73.6ms	remaining: 18m 24s
250:	learn: 0.7909160	test: 0.7841527	best: 0.7841720 (249)	total: 15.4s	remaining: 15m 6s
500:	learn: 0.7998943	test: 0.7863531	best: 0.7863531 (500)	total: 30.5s	remaining: 14m 43s
750:	learn: 0.8069620	test: 0.7870249	best: 0.7870412 (749)	total: 45.5s	remaining: 14m 23s
1000:	learn: 0.8131877	test: 0.7877793	best: 0.7877793 (1000)	total: 1m	remaining: 14m 12s
1250:	learn: 0.8188098	test: 0.7876726	best: 0.7878162 (1025)	total: 1m 16s	remaining: 14m
1500:	learn: 0.8243219	test: 0.7875915	best: 0.7878162 (1025)	total: 1m 31s	remaining: 13m 46s
bestTest = 0.7878162265
bestIteration = 1025
Shrink model to first 1026 iterations.
0:	learn: 0.8323758	test: 0.8285008	best: 0.8285008 (0)	total: 61.9ms	remaining: 15m 28s
250:	learn: 0.8568825	test: 0.8569839	best: 0.8570051 (249)	total: 15.5s	remaining: 15m 13s
500:	learn: 0.8629215	test: 0.8596426	best: 0.8596426 (500)	total: 31.1s	remaining: 14m 59s
75

In [0]:
for key, clf in clfs.items():
    test[key] = clf.predict_proba(big.loc[test.index])[:, 1]

In [0]:
test.to_csv(path.format('otvet.csv'))

In [0]:
for i in clfs:
  clfs[i].save_model(f'{i}_model', format='cbm')